In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm  # Para barra de progresso

# Configurações
INPUT_IMAGE_DIR = "raw_data/artDataset"
OUTPUT_IMAGE_DIR = "processed_data/artDataset_clean"
CSV_PATH = "raw_data/artDataset.csv"
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

# Carregar o CSV para manter referência dos IDs
df = pd.read_csv(CSV_PATH)

# Função aprimorada para remoção de molduras e fundos brancos
def remove_frame_and_background(img_path, output_path, threshold=220, margin=10):
    try:
        img = cv2.imread(img_path)
        if img is None:
            return False

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 1. Binarização para detectar fundos brancos
        _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)

        # 2. Operações morfológicas para remover ruídos
        kernel = np.ones((5, 5), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

        # 3. Encontrar o maior contorno (obra de arte)
        contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return False

        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # 4. Recortar com margem de segurança
        cropped = img[
            max(0, y-margin):min(img.shape[0], y+h+margin),
            max(0, x-margin):min(img.shape[1], x+w+margin)
        ]

        cv2.imwrite(output_path, cropped)
        return True

    except Exception as e:
        print(f"Erro ao processar {img_path}: {str(e)}")
        return False

# Processar todas as imagens com barra de progresso
success_count = 0
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_filename = f"image_{row['Unnamed: 0']+1}.png"  # Assumindo formato 'ID.jpg'
    input_path = os.path.join(INPUT_IMAGE_DIR, img_filename)
    output_path = os.path.join(OUTPUT_IMAGE_DIR, img_filename)

    if os.path.exists(input_path):
        if remove_frame_and_background(input_path, output_path):
            success_count += 1
    else:
        print(f"Arquivo não encontrado: {input_path}")

print(f"\nProcessamento concluído. {success_count}/{len(df)} imagens processadas com sucesso.")

# Atualizar CSV com caminhos das novas imagens (opcional)
df['image_clean_path'] = df['Unnamed: 0'].apply(lambda x: f"artDataset_clean/{x}.png")
df.to_csv("processed_data/artDataset_clean.csv", index=False)

100%|██████████| 754/754 [00:12<00:00, 58.96it/s]


Processamento concluído. 754/754 imagens processadas com sucesso.


In [3]:
print(df.columns.tolist())

['Unnamed: 0', 'price', 'artist', 'title', 'yearCreation', 'signed', 'condition', 'period', 'movement']
